In [7]:
import pandas as pd
from pulp import *

In [6]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.4 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [10]:
stock = pd.read_csv("../Data/data/inventory.csv")
predicted_demand = pd.read_csv("../data/forecasted_demand.csv")
transport_cost = pd.read_csv("../Data/data/transport_costs.csv")

In [ ]:
date = stock['date'].max()
stock_today = stock[stock['date'] == date].groupby('warehouse')['stock_level'].sum().to_dict()
demand_today = demand[demand['date'] == date].groupby('warehouse')['demand_units'].sum().to_dict()
cost_dict = costs.set_index(['from_warehouse', 'to_warehouse'])['cost_per_unit'].to_dict()

# Define LP model
model = LpProblem("SupplyChainOptimizer", LpMinimize)

warehouses = list(stock_today.keys())
transfers = [(i, j) for i in warehouses for j in warehouses if i != j]

# Decision variables
x = LpVariable.dicts("Transfer", transfers, lowBound=0, cat='Integer')

# Objective: Minimize total transfer cost
model += lpSum([x[i] * cost_dict[i] for i in transfers])

# Constraints: Don’t overdraw stock, meet demand
for w in warehouses:
    outgoing = lpSum([x[(w, j)] for j in warehouses if w != j])
    incoming = lpSum([x[(i, w)] for i in warehouses if i != w])
    net_stock = stock_today.get(w, 0) + incoming - outgoing
    demand_needed = demand_today.get(w, 0)
    model += net_stock >= demand_needed

# Solve
model.solve()
print("Status:", LpStatus[model.status])
for var in model.variables():
    if var.varValue > 0:
        print(var.name, "=", var.varValue)